In [5]:
import pandas as pd
import numpy as np

UC3MAL201 Machine Learning Session 11 Tutorial 11 Leon Eriksen Helgeland -
First written 09.09.2020

# Neural Network - with Keras

In [6]:
dataset_file_name = 'mnist_train.csv'
dataset_directory = '~/GitHub/BigData/dataset11'

In [7]:
dataframe = pd.read_csv(f'{dataset_directory}/{dataset_file_name}')
df = dataframe

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59999 entries, 0 to 59998
Columns: 785 entries, 5 to 0.617
dtypes: int64(785)
memory usage: 359.3 MB


In [9]:
df.describe()

,5,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,...,0.608,0.609,0.610,0.611,0.612,0.613,0.614,0.615,0.616,0.617
count,59999.000000,59999.0,59999.0,59999.0,59999.0,59999.0,59999.0,59999.0,59999.0,59999.0,...,59999.000000,59999.000000,59999.000000,59999.000000,59999.000000,59999.000000,59999.0,59999.0,59999.0,59999.0
mean,4.453924,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.200437,0.088868,0.045634,0.019284,0.015117,0.002000,0.0,0.0,0.0,0.0
std,2.889294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.042522,3.956222,2.839868,1.686784,1.678297,0.346603,0.0,0.0,0.0,0.0
min,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
25%,2.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
50%,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
75%,7.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
max,9.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,254.000000,254.000000,253.000000,253.000000,254.000000,62.000000,0.0,0.0,0.0,0.0


## Dataset preparation
Rescale the data to values between 0.01 and 1, by multiplying each pixel value in the dataset by (0.99/255) and adding 0.01 to the result to get a new pixel value.

In [10]:
for i in range(0,256):
    print(i,(i*(0.99/255))+0.01)

0 0.01
1 0.01388235294117647
2 0.01776470588235294
3 0.02164705882352941
4 0.025529411764705884
5 0.029411764705882353
6 0.03329411764705882
7 0.0371764705882353
8 0.041058823529411766
9 0.044941176470588234
10 0.0488235294117647
11 0.05270588235294118
12 0.05658823529411765
13 0.060470588235294116
14 0.06435294117647058
15 0.06823529411764706
16 0.07211764705882352
17 0.076
18 0.07988235294117646
19 0.08376470588235294
20 0.0876470588235294
21 0.09152941176470587
22 0.09541176470588235
23 0.09929411764705881
24 0.10317647058823529
25 0.10705882352941176
26 0.11094117647058822
27 0.1148235294117647
28 0.11870588235294117
29 0.12258823529411764
30 0.1264705882352941
31 0.1303529411764706
32 0.13423529411764706
33 0.13811764705882354
34 0.14200000000000002
35 0.14588235294117646
36 0.14976470588235294
37 0.15364705882352941
38 0.1575294117647059
39 0.16141176470588237
40 0.1652941176470588
41 0.1691764705882353
42 0.17305882352941176
43 0.17694117647058824
44 0.18082352941176472
45 0.184

In [36]:
counter = 0
rand_list = []
for i in df.columns:
    #print(i)
    counter+=1
    try:
        print(f'Line {counter}:',df.value_counts(df.columns[counter-1]))
    except IndexError:
        rand_list.append(i)
print(rand_list)


Line 677: 0.517
0      59903
7          7
1          5
6          5
4          4
       ...  
138        1
163        1
60         1
55         1
185        1
Length: 63, dtype: int64
Line 678: 136.1
0      59669
253       10
3          8
22         6
252        6
       ...  
79         1
80         1
139        1
84         1
54         1
Length: 165, dtype: int64
Line 679: 253.51
0      59063
253       59
252       29
1         21
3         18
       ...  
90         1
175        1
39         1
176        1
110        1
Length: 223, dtype: int64
Line 680: 253.52
0      57745
253      137
252       80
7         49
254       47
       ...  
240        1
106        1
242        1
184        1
204        1
Length: 256, dtype: int64
Line 681: 253.53
0      55670
253      351
252      230
254      107
255       75
       ...  
163        3
198        3
209        2
115        2
182        2
Length: 256, dtype: int64
Line 682: 212
0      52833
253      783
252      448
254      243
255   

In [17]:
df.columns

Index(['5', '0', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8',
       ...
       '0.608', '0.609', '0.610', '0.611', '0.612', '0.613', '0.614', '0.615',
       '0.616', '0.617'],
      dtype='object', length=785)

In [26]:
df2 = df.drop(df.columns[0], axis=1)

In [44]:
for row, column in df2:
    print(row, column)

# for i in x:
#     if i == 255:
#         i = 0.999
#     else:
#         i = (i*(0.99/255))+0.01

ValueError: not enough values to unpack (expected 2, got 1)

In [43]:
dtypes_list = set()
for i in df.dtypes:
    print(i)
    dtypes_list.add(i)
print(dtypes_list)

int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int6